In [5]:
#pip install google-generativeai
import sys
sys.setrecursionlimit(100000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [6]:
import warnings
warnings.filterwarnings("ignore")


In [7]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel("gemini-2.0-flash")



True

In [8]:
import os
from typing import List,TypedDict
from langgraph.graph import StateGraph,END
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [9]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
emb = HuggingFaceBgeEmbeddings(model_name="thenlper/gte-small")

In [10]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

# 1. Define the Root URL (Where to start)
# Note: LangChain docs structure changes often. Ensure this is the active root.
url = "https://python.langchain.com/v0.2/docs/introduction/" 

# 2. Configure the Loader
loader = RecursiveUrlLoader(
    url=url,
    max_depth=2, # How deep to dig? (Root -> Child -> Grandchild)
    # This specifically filters for the main content div to avoid navbars/footers
    extractor=lambda x: Soup(x, "html.parser").text 
)

# 3. Load the docs
document = loader.load()

print(f"Loaded {len(document)} documents.")

Loaded 47 documents.


In [11]:
print(document[6])


page_content='LangChain overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this page Create an agent Core benefitsLangChain overviewCopy pageCopy pageLangChain v1.x is now available!For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide.If you encounter

In [19]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context. 
Also if any code snippet is available please provide it.
Always answer for more then 100 words. 
<context>
{context}
</context>
Question : {input}""")

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [14]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

docs = splitter.split_documents(document)

In [15]:
vectorstore = FAISS.from_documents(docs,embedding=emb)

In [16]:
retriever = vectorstore.as_retriever(k=3)


In [24]:
context_docs = retriever.invoke("Give to me in detail about how to use agents in langchain")
context = format_docs(context_docs)

final_prompt = prompt.format(context=context, input="Give to me in detail about how to use agents in langchain")

response = model.generate_content(final_prompt)
print(response.text)


LangChain simplifies the process of creating agents and applications powered by Large Language Models (LLMs). Here's a breakdown of how to use agents in LangChain, based on the provided context.

LangChain offers a streamlined approach, enabling you to build agents with relatively few lines of code. Specifically, the documentation mentions achieving this in under 10 lines of code. This ease of use stems from LangChain's provision of a pre-built agent architecture and model integrations. This means you don't have to build the underlying structure from scratch or worry extensively about connecting to various LLM providers.

To get started, LangChain allows you to connect to different LLM providers such as OpenAI, Anthropic, and Google. By leveraging the pre-built agent architecture and model integrations, developers can seamlessly incorporate LLMs into their agents and applications.

However, for more advanced scenarios requiring deterministic and agentic workflows, heavy customization, 

In [23]:
print(context_docs)

[Document(id='fd58530d-cb93-4ed9-870d-e91dcf9dc8ba', metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'content_type': 'text/html; charset=utf-8', 'title': 'LangChain overview - Docs by LangChain', 'language': 'en'}, page_content='We recommend you use LangChain if you want to quickly build agents and autonomous applications. Use LangGraph, our low-level agent orchestration framework and runtime, when you have more advanced needs that require a combination of deterministic and agentic workflows, heavy customization, and carefully controlled latency.\nLangChain agents are built on top of LangGraph in order to provide durable execution, streaming, human-in-the-loop, persistence, and more. You do not need to know LangGraph for basic LangChain agent usage.\n\u200b Create an agent\nCopy# pip install -qU langchain "langchain[anthropic]"\nfrom langchain.agents import create_agent'), Document(id='45f058f0-1cf0-49a8-8bc1-f435c653111d', metadata={'source': 'https://pytho